# Prequel: searching for data from ISTAT

This is a Notebook done a-posteriori where I extensively investigate which data are available from ISTAT, 
as I could not find a comprehensive list.

In [1]:
import warnings 
from istatapi import discovery, retrieval
import numpy as np
import pandas as pd
import requests

import plotly.io as pio
pio.renderers.default = 'notebook'

warnings.filterwarnings('ignore')
requests.urllib3.disable_warnings()

! pip show istatapi

Name: istatapi
Version: 1.0.0
Summary: Python API for ISTAT (The Italian National Institute of Statistics)
Home-page: https://github.com/Attol8/istatapi
Author: Jacopo Attolini
Author-email: jacopoattolini@gmail.com
License: Apache Software License 2.0
Location: /Users/danieleongari/opt/anaconda3/envs/py310/lib/python3.10/site-packages
Requires: fastcore, matplotlib, pandas, pyopenssl, requests
Required-by: 


In [2]:
all_datasets = discovery.search_dataset("")
display(all_datasets)

,df_id,version,df_description,df_structure_id
0,101_1015,1.3,Crops,DCSP_COLTIVAZIONI
1,101_1030,1.0,"PDO, PGI and TSG quality products",DCSP_DOPIGP
2,101_1033,1.0,slaughtering,DCSP_MACELLAZIONI
3,101_1039,1.2,Agritourism - municipalities,DCSP_AGRITURISMO_COM
4,101_1077,1.0,"PDO, PGI and TSG products: operators - munici...",DCSP_DOPIGP_COM
...,...,...,...,...
504,97_953,1.1,Environmental protection expenditure,DCCN_SPESAPROTAMB
505,98_1066,1.0,Productivity measures - Accounts in the 2014 v...,DCCN_PRODUTTIVITA_B14
506,98_1067,1.0,Productivity measures - Accounts in the 2011 v...,DCCN_PRODUTTIVITA_B11
507,98_197,1.3,Productivity measures,DCCN_PRODUTTIVITA


In [3]:
selected_datasets = (
    all_datasets
    .query('df_description.str.contains("population|balance|demographic|mort|births|death|fertility|life|killed", case=False)')
    .query('~df_description.str.contains("balance sheet")')
    .sort_values('df_structure_id', ascending=True)
)

count=0
for _, row in selected_datasets.iterrows():
    print(f'{count+1:2d} - v{row["version"]} {row["df_structure_id"]}  - {row["df_description"]}')
    count+=1

 1 - v1.1 DCCN_OCCNSEC2010_B14  - Regular, irregular employment and population - editions from October 2014 to April 2019
 2 - v1.1 DCCN_OCCTSEC2010_B14  - Regular, irregular employment and population - editions from October 2012 to December 2018
 3 - v1.1 DCCV_AVQ_FAMIGLIE  - Aspects of daily life - Household
 4 - v1.3 DCCV_AVQ_PERSONE  - Aspects of daily life
 5 - v1.1 DCCV_AVQ_PERSONE1  - Aspects of daily life - Persons
 6 - v1.3 DCCV_CONDGEO1  - Persons convicted by final judgement - Demographic characteristics
 7 - v1.2 DCCV_DETENUTI  - Prison population held in institutions for adults in Italy
 8 - v1.1 DCCV_DIPL_DEM_FORM  - High school graduates - socio-demographic characteristics and curriculum
 9 - v1.2 DCCV_INATTIV1  - Inactive  population
10 - v1.0 DCCV_INATTIV1_UNT2020  - Inactive  population - previous regulation (until 2020)
11 - v1.3 DCCV_INATTIVDE1  - Inactive population - quarterly seasonally adjusted data
12 - v1.0 DCCV_INATTIVDE1_UNT2020  - Inactive population - quar

In [6]:
interesting_dss = ["DCIS_POPRES1", "DCIS_POPORESBIL1", "DCIS_RICPOPRES2011",  "DCIS_PREVDEM1", "DCIS_POPSTRBIL1", "DCIS_FECONDITA1", "DCIS_DECESSI"] # "DCIS_RICSTAT" failed

for ds_structure_id in interesting_dss:
    print(f"\n############ {ds_structure_id} ################################################################")
    print("Description:", all_datasets.query(f'df_structure_id == "{ds_structure_id}"').iloc[0].df_description)
    ds = discovery.DataSet(dataflow_identifier=ds_structure_id)
    ds_info = ds.dimensions_info()
    display(ds_info)
    for col in ds_info.dimension:
        print(f" >> {col}")
        display(ds.get_dimension_values(col).sort_values("values_ids").reset_index(drop=True))


############ DCIS_POPRES1 ################################################################
Description: Resident population  on 1st January


,dimension,dimension_ID,description
0,FREQ,CL_FREQ,Frequency
1,ETA,CL_ETA1,Age class
2,ITTER107,CL_ITTER107,Territory
3,SESSO,CL_SEXISTAT1,Gender
4,STACIVX,CL_STATCIV2,Marital status
5,TIPO_INDDEM,CL_TIPO_DATO15,Data type 15


 >> FREQ


,values_ids,values_description
0,A,annual


 >> ETA


...

,values_ids,values_description
0,1,never married
1,12,legally separated
2,2,married
3,3,divorced
4,4,widowed
5,98,n.a.
6,99,total


 >> TIPO_DATO


,values_ids,values_description
0,DEATH,deaths
1,DEATHRATE,death rate (per thousand inhabitant)
2,DEATHRATEAGE,age-specific daeth rate (per thousand inhabitant)
3,LEXISPOINT,lexis point
4,LIFEEXP0,life expectancy at birth
5,LIFEEXP65,life expectancy at 65
6,LIFEEXP85,life expectancy at 85
7,MEANAGEDH,mean age at death
8,PROBDEATH0Y,probability of death at 0 year


 >> TITOLO_STUDIO


,values_ids,values_description
0,3,"primary school certificate, no educational degree"
1,4,lower secondary school certificate
2,44,first degree (new programme)
3,48,"long university degree (old programme), long d..."
4,57,with post-university specialization or post-d...
5,7,upper and post secondary
6,99,total


## Conclusions
- `DCIS_POPRES1`
- `DCIS_POPORESBIL1`
- `DCIS_RICPOPRES2011`
- `DCIS_PREVDEM1`
- `DCIS_POPSTRBIL1`
- `DCIS_FECONDITA1`
- `DCIS_DECESSI`